In [22]:
import numpy as np
from astropy.time import Time
import astropy.units as u
from astropy.coordinates import solar_system_ephemeris, EarthLocation, get_icrs_coordinates, AltAz
from astropy.coordinates import get_body_barycentric, get_body, get_moon
t = Time.now()
loc = EarthLocation.of_address('Pordenone, Italy')

In [23]:
mizar = get_icrs_coordinates('Mizar')
alcor = get_icrs_coordinates('Alcor')
aa = AltAz(location=loc, obstime=t)
mizar.transform_to(aa)

<SkyCoord (AltAz: obstime=2020-09-19 08:58:29.326012, location=(4333831.2033362, 973470.0255359, 4561868.2083341) m, pressure=0.0 hPa, temperature=0.0 deg_C, relative_humidity=0.0, obswl=1.0 micron): (az, alt) in deg
    (54.95186594, 54.71477923)>

In [21]:
mizar.separation(alcor)

<Angle 0.19682972 deg>